Number of identified patterns per class. (Fig. 19, Supplementary Material)

In [ ]:
# default print properties
multiplier = 2

pixel_cm_ration = 36.5

width_full = int(13.95 * pixel_cm_ration) * multiplier
width_half = int(13.95/2 * pixel_cm_ration) * multiplier

height_default_1 = int(3.5 * pixel_cm_ration) * multiplier
height_default_2 = int(7 * pixel_cm_ration) * multiplier

# margins in pixel
top_margin = 10 * multiplier 
left_margin = 35 * multiplier 
right_margin = 0 * multiplier 
bottom_margin = 20 * multiplier 

font_size = 10 * multiplier 
font_family='Times New Roman'

line_width = 2 * multiplier 

In [ ]:
# Define and load data
import autodisc as ad
import ipywidgets
import plotly
import numpy as np
import collections
plotly.offline.init_notebook_mode(connected=True)

data_filters = collections.OrderedDict()
data_filters['none'] = []
data_filters['non dead'] = ('classifier_dead.data', '==', False)
data_filters['animals (non div)'] = (('classifier_diverging.data', '==', 0), 'and', ('classifier_animal.data', '==', True))
data_filters['non animals (non div)'] = ((('classifier_dead.data', '==', False), 'and', ('classifier_animal.data', '==', False)), 'and', ('classifier_diverging.data', '==', 0))
data_filters['animals (all)'] = ('classifier_animal.data', '==', True)
data_filters['non animals (all)'] = (('classifier_dead.data', '==', False), 'and', ('classifier_animal.data', '==', False))

org_experiment_definitions = dict()

org_experiment_definitions['main_paper'] = [
     
    dict(id = '1',
         directory = '../experiments/experiment_000001',
         name = 'Random',
         is_default = True),

    dict(id = '109',
         directory = '../experiments/experiment_000109',
         name = 'IMGEP-HGS',
         is_default = True),
    
    dict(id = '202',
         directory = '../experiments/experiment_000202',
         name = 'IMGEP-PGL',
         is_default = True),

    dict(id = '302',
         directory = '../experiments/experiment_000302',
         name = 'IMGEP-OGL',
         is_default = True),
]




repetition_ids = list(range(10))

# define names and load the data
experiment_name_format = '<name>' # <id>, <name>

#global experiment_definitions
experiment_definitions = []
experiment_statistics = []

current_experiment_list = 'main_paper'

experiment_definitions = []
for org_exp_def in org_experiment_definitions[current_experiment_list]:
    new_exp_def = dict()
    new_exp_def['directory'] = org_exp_def['directory']
    if 'is_default' in org_exp_def:
        new_exp_def['is_default'] = org_exp_def['is_default']

    if 'name' in org_exp_def:
        new_exp_def['id'] = ad.gui.jupyter.misc.replace_str_from_dict(experiment_name_format, {'id': org_exp_def['id'], 'name': org_exp_def['name']})
    else:
        new_exp_def['id'] = ad.gui.jupyter.misc.replace_str_from_dict(experiment_name_format, {'id': org_exp_def['id']})

    experiment_definitions.append(new_exp_def)

experiment_statistics = dict()
for experiment_definition in experiment_definitions:
    experiment_statistics[experiment_definition['id']] = ad.gui.jupyter.misc.load_statistics(experiment_definition['directory'])
       
        
##############################################################################        
## Collect Data for categories

# function to calcuate ratios
def calc_binary_ratio(array):
    return np.sum(array) / np.sum(~np.isnan(array))

classifier_data = dict()
for experiment_definition in experiment_definitions: 

    experiment_id = experiment_definition['id']

    try:
    
        dead_data = experiment_statistics[experiment_id]['classifier_dead']['data']
        dead_ratio = experiment_statistics[experiment_id]['classifier_dead']['ratio'] * 100
        
        non_dead_data = (experiment_statistics[experiment_id]['classifier_dead']['data'] == False)
        non_dead_ratio = (1 - experiment_statistics[experiment_id]['classifier_dead']['ratio']) * 100

        animal_data = experiment_statistics[experiment_id]['classifier_animal']['data']
        animal_ratio = experiment_statistics[experiment_id]['classifier_animal']['ratio'] * 100

        non_animal_data = np.full(dead_data.shape, True)
        non_animal_data[dead_data] = False
        non_animal_data[animal_data] = False
        non_animal_ratio = np.apply_along_axis(calc_binary_ratio, 1, non_animal_data) * 100

        diverging_data = np.full(experiment_statistics[experiment_id]['classifier_diverging']['data'].shape, True)
        diverging_data[experiment_statistics[experiment_id]['classifier_diverging']['data'] == 0] = False
        diverging_ratio = np.apply_along_axis(calc_binary_ratio, 1, diverging_data)

        diverging_animal_data = animal_data & diverging_data
        diverging_animal_ratio = np.apply_along_axis(calc_binary_ratio, 1, diverging_animal_data)

        non_diverging_animal_data = animal_data & ~diverging_data
        non_diverging_animal_ratio = np.apply_along_axis(calc_binary_ratio, 1, non_diverging_animal_data)

        diverging_non_animal_data = non_animal_data & diverging_data
        diverging_non_animal_ratio = np.apply_along_axis(calc_binary_ratio, 1, diverging_non_animal_data)

        non_diverging_non_animal_data = non_animal_data & ~diverging_data
        non_diverging_non_animal_ratio = np.apply_along_axis(calc_binary_ratio, 1, non_diverging_non_animal_data)


        classifier_data[experiment_id] = dict(     
                                dead_data = dead_data,
                                dead_ratio = dead_ratio,

                                non_dead_data = non_dead_data,
                                non_dead_ratio = non_dead_ratio,

                                animal_data = animal_data,
                                animal_ratio = animal_ratio,

                                non_animal_data = non_animal_data,
                                non_animal_ratio = non_animal_ratio,

                                diverging_data = diverging_data,
                                diverging_ratio = diverging_ratio,

                                diverging_animal_data = diverging_animal_data,
                                diverging_animal_ratio = diverging_animal_ratio,

                                non_diverging_animal_data = non_diverging_animal_data,
                                non_diverging_animal_ratio = non_diverging_animal_ratio,

                                diverging_non_animal_data = diverging_non_animal_data,
                                diverging_non_animal_ratio = diverging_non_animal_ratio,

                                non_diverging_non_animal_data = non_diverging_non_animal_data,
                                non_diverging_non_animal_ratio = non_diverging_non_animal_ratio,
            )
        
    except Exception as e:
        raise RuntimeError('Error for Experiment {!r}!'.format(experiment_id) ) from e        
        

# Number of identified classes

In [ ]:
# Plotting
config = dict(
    plot_type = 'plotly_box',
    layout = dict(
        yaxis= dict(
            title='percentage',
            range = [0, 81],
            showline = True,
            linewidth = 1,
            zeroline=False,
            ),
        xaxis=dict(
            showline = True,
            linewidth = 1,
            zeroline=False,
            ),
        font = dict(
            family=font_family, 
            size=font_size, 
            ),
        width = width_full, # in cm
        height = height_default_2, # in cm
        
        margin = dict(
            l=left_margin, #left margin in pixel
            r=right_margin, #right margin in pixel
            b=bottom_margin, #bottom margin in pixel
            t=top_margin,  #top margin in pixel
            ),
        
        
        updatemenus=[],
    ),
    init_mode='all',
    trace_labels = ['dead', 'non-animals', 'animals'],

    
    default_trace=dict(
        boxmean=True,
    ),
    
    traces = [
        dict(marker=dict(color='rgb(0,0,0)')),
        dict(marker=dict(color='rgb(0,158,115)')),
        dict(marker=dict(color='rgb(230,159,0)')),
    ]
)


fig = ad.gui.jupyter.plot_barbox_per_datasource(experiment_ids=[exp_def['id'] for exp_def in experiment_definitions],
                                                repetition_ids=repetition_ids, 
                                                data=classifier_data, 
                                                data_source=['dead_ratio', 'non_animal_ratio', 'animal_ratio'],
                                                config=config)
